# SOO Experiment design

We show how to design your experiment when you only care about 1 thing - minimising the loss of the experiment assuming none of the sensors fail. This is single objective optimisation.

In [ ]:
%matplotlib inline
import sys
import os

# Sort the paths out to run from this file
parent_path = os.path.dirname(os.getcwd())
src_path = os.path.join(os.path.sep,parent_path, 'src')
sys.path.append(src_path)
print(src_path)

## Design a 2D scalar field experiment with no assumptions with SOO

We load the correct field file and grid file that we generated in step 1.

We then define the region that the sensors can be placed in (no less than 2 mm from the monoblock edges).

We setup the sensor suite after defining that the sensors will have zero systematic error at any temperature. The sensor suite requires an input field type to allow it to predict the values of an entire field based off the sensor values it is given.

We setup the experiment and define its optimiser and use the grid of comparison positions defined in step 1. 

Finally we display the results.

Things to change
1. Names of the `field_temp.obj` and `grid_temp.obj` if you want new fields and grids that you defined in step 1
2. The sensor bounds
3. The systematic error function does not have to be `def f(x): return 0`
4. You can add in symmetry assumptions (see next example)
5. The optimise can be changed (note that this is single objective optimisation to minimise the loss only so only single objective optimisers can be used)
6. Swap out the model used in the field of the `SensorSuite` for a different ML regression model to see what kind of job that does.

In [ ]:
from run_me import *


# Load any objects necessary
pickle_manager = PickleManager()
graph_manager = GraphManager()
true_temp_field = pickle_manager.read_file('simulation', 'field_temp.obj')
grid = pickle_manager.read_file('simulation', 'grid_temp.obj')

bounds = true_temp_field.get_bounds()
sensor_bounds = bounds+np.array([[1, 1], [-1, -1]])*0.002

# Setup the sensor suite
def f(x): return 0
sensor = PointSensor(0, f, 0, [-5000, 5000])
sensors = np.array([sensor]*5)
sensor_suite = SensorSuite(
    ScalarField(RBFModel, bounds, true_temp_field.get_dim()), 
    sensors,
    symmetry=[]
)

# Setup the experiment
optimiser = PSOOptimiser('00:00:10')
experiment = Experiment(
    true_temp_field,
    grid,
    optimiser
)
experiment.plan_soo(
    sensor_suite,
    sensor_bounds
)
res = experiment.design()
proposed_layout, true_temps, model_temps, sensor_values = experiment.get_plotting_arrays(res.X)


# Display the results
graph_manager.build_optimisation(
    res.history
)
graph_manager.draw()
graph_manager.build_2D_compare(
    grid,
    proposed_layout,
    true_temps,
    model_temps
)
graph_manager.draw()
graph_manager.build_3D_compare(
    grid,
    true_temps,
    model_temps
)
graph_manager.draw()

## Design a 2D scalar field experiment with symmetry assumptions with SOO

Now we add in symmetry assumptions. Set the x-value of the symmetry manager and then force symmetry in the line of that x value (horizontal reflection). And/or set the y-value and then force symmetry in the line of that y-value (vertical reflection). And/or set the grad-value and force symmetry in the line of that gradient of the form y=mx (line reflection). 

By combining these symmetries you can get some pretty interesting looking fields. You can also do the same for 1D fields as I'll show later.

In [ ]:
from run_me import *


# Load any objects necessary
pickle_manager = PickleManager()
graph_manager = GraphManager()
true_temp_field = pickle_manager.read_file('simulation', 'field_temp.obj')
grid = pickle_manager.read_file('simulation', 'grid_temp.obj')

bounds = true_temp_field.get_bounds()
sensor_bounds = bounds+np.array([[1, 1], [-1, -1]])*0.002

# Setup the symmetry
symmetry_manager = SymmetryManager()
symmetry_manager.set_2D_x(np.mean(bounds[:, 0]))

# Setup the sensor suite
def f(x): return 0
sensor = PointSensor(0, f, 0, [-5000, 5000])
sensors = np.array([sensor]*5)
sensor_suite = SensorSuite(
    ScalarField(RBFModel, bounds, true_temp_field.get_dim()), 
    sensors,
    symmetry=[symmetry_manager.reflect_2D_horiz]
)

# Setup the experiment
optimiser = PSOOptimiser('00:00:10')
experiment = Experiment(
    true_temp_field,
    grid,
    optimiser
)
experiment.plan_soo(
    sensor_suite,
    sensor_bounds
)
res = experiment.design()
proposed_layout, true_temps, model_temps, sensor_values = experiment.get_plotting_arrays(res.X)


# Display the results
graph_manager.build_optimisation(
    res.history
)
graph_manager.draw()
graph_manager.build_2D_compare(
    grid,
    proposed_layout,
    true_temps,
    model_temps
)
graph_manager.draw()
graph_manager.build_3D_compare(
    grid,
    true_temps,
    model_temps
)
graph_manager.draw()

## Design a 1D scalar field experiment with no assumptions SOO

Now we'll do the same for a 1D experiment. Note that we can get away with using the line data as the temperature field on the 'right' monoblock face is uniform horizontally.



In [ ]:
from run_me import *


# Load any objects necessary
pickle_manager = PickleManager()
graph_manager = GraphManager()
true_temp_field = pickle_manager.read_file('simulation', 'field_temp_line.obj')
grid = pickle_manager.read_file('simulation', 'line_temp.obj')

bounds = true_temp_field.get_bounds()
sensor_bounds = bounds + np.array([[1], [-1]])*0.002


# Setup the sensor suite
def f(x): return 0
sensor = PointSensor(0, f, 0, [-5000, 5000])
sensors = np.array([sensor]*5)
sensor_suite = SensorSuite(
    ScalarField(RBFModel, bounds, true_temp_field.get_dim()), 
    sensors,
    symmetry=[]
)


# Setup the experiment
optimiser = PSOOptimiser('00:00:10')
experiment = Experiment(
    true_temp_field,
    grid,
    optimiser
)
experiment.plan_soo(
    sensor_suite,
    sensor_bounds
)
res = experiment.design()
proposed_layout, true_temps, model_temps, sensor_values = experiment.get_plotting_arrays(res.X)


# Display the results
graph_manager.build_optimisation(
    res.history
)
graph_manager.draw()
graph_manager.build_1D_compare(
    grid,
    proposed_layout,
    sensor_values,
    true_temps,
    model_temps,
)
graph_manager.draw()

## Design a 1D scalar experiment with assumptions SOO

We'll add in some symmetry assumptions to our code, see below. Notice that as the field does not actually have a line of symmetry at x=0.015, it leads to highly innacurate models of the field.


In [ ]:
from run_me import *


# Load any objects necessary
pickle_manager = PickleManager()
graph_manager = GraphManager()
true_temp_field = pickle_manager.read_file('simulation', 'field_temp_line.obj')
grid = pickle_manager.read_file('simulation', 'line_temp.obj')

bounds = true_temp_field.get_bounds()
sensor_bounds = bounds + np.array([[1], [-1]])*0.002

# Add the symmetry
symmetry_manager = SymmetryManager()
symmetry_manager.set_1D_x(0.015)

# Setup the sensor suite
def f(x): return 0
sensor = PointSensor(0, f, 0, [-5000, 5000])
sensors = np.array([sensor]*5)
sensor_suite = SensorSuite(
    ScalarField(GPModel, bounds, true_temp_field.get_dim()), 
    sensors,
    symmetry=[symmetry_manager.reflect_1D]
)


# Setup the experiment
optimiser = PSOOptimiser('00:00:10')
experiment = Experiment(
    true_temp_field,
    grid,
    optimiser
)
experiment.plan_soo(
    sensor_suite,
    sensor_bounds
)
res = experiment.design()
proposed_layout, true_temps, model_temps, sensor_values = experiment.get_plotting_arrays(res.X)


# Display the results
graph_manager.build_optimisation(
    res.history
)
graph_manager.draw()
graph_manager.build_1D_compare(
    grid,
    proposed_layout,
    sensor_values,
    true_temps,
    model_temps,
)
graph_manager.draw()

## Design a 1D experiment with Thermocouple sensors


Now we've swapped the sensors for thermocouples which are noisy, systematically wrong, and have an imperfect range.

In [ ]:
from run_me import *

# Load any objects necessary
pickle_manager = PickleManager()
graph_manager = GraphManager()
true_temp_field = pickle_manager.read_file('simulation', 'field_temp_line.obj')
grid = pickle_manager.read_file('simulation', 'line_temp.obj')

bounds = true_temp_field.get_bounds()
sensor_bounds = bounds + np.array([[1], [-1]])*0.002

# Setup the sensor suite
temps = pickle_manager.read_file('sensors', 'k-type-T.obj')
voltages = pickle_manager.read_file('sensors', 'k-type-V.obj')
sensor = Thermocouple(temps, voltages)
sensors = np.array([sensor]*5)
sensor_suite = SensorSuite(
    ScalarField(RBFModel, bounds, true_temp_field.get_dim()), 
    sensors,
    symmetry=[]
)

# Setup the experiment
optimiser = PSOOptimiser('00:00:10')
experiment = Experiment(
    true_temp_field,
    grid,
    optimiser
)
experiment.plan_soo(
    sensor_suite,
    sensor_bounds
)
res = experiment.design()
proposed_layout, true_temps, model_temps, sensor_values = experiment.get_plotting_arrays(res.X)


# Display the results
graph_manager.build_optimisation(
    res.history
)
graph_manager.draw()
graph_manager.build_1D_compare(
    grid,
    proposed_layout,
    sensor_values,
    true_temps,
    model_temps,
)
graph_manager.draw()